In [1]:
!pip install -qqq -U transformers datasets accelerate peft trl bitsandbytes wandb --progress-bar off

In [ ]:
pip install cupy-cuda11x==12.0.0

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
pip install transformers==4.30.0 accelerate==0.12.0

In [ ]:
!pip uninstall transformers -Y
!pip install transformers 

In [3]:
import gc #garbade collector library
import os

import torch
import wandb
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from trl import ORPOConfig, ORPOTrainer, setup_chat_format

# Model
base_model = "meta-llama/Llama-2-7b-hf"
new_model = "OrpoLlama-2-7b"

2024-06-28 09:30:51.368372: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-28 09:30:51.368512: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-28 09:30:51.499199: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# Set torch dtype and attention implementation
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    torch_dtype = torch.bfloat16
    attn_implementation = "flash_attention_2"
else:
    torch_dtype = torch.float16
    attn_implementation = "eager"

print(attn_implementation)

eager


In [5]:
from accelerate import Accelerator

current_device = Accelerator().process_index


In [6]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)

# Load model
model = AutoModelForCausalLM.from_pretrained(
     base_model,
     quantization_config=bnb_config,
     device_map = {"":current_device},
     attn_implementation=attn_implementation
)
model, tokenizer = setup_chat_format(model, tokenizer)
model = prepare_model_for_kbit_training(model)
#tokenizer = setup_chat_format(tokenizer)

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

## the dataset is a crucial part of the training process and we need to make sure that it in the right form

In [7]:
import pandas as pd
from datasets import Dataset
df=pd.read_excel("/kaggle/input/orpo-dataset/ORPO_dataset -1.xlsx")

In [8]:
df = df.rename(columns={'question': 'prompt',"Chosen":"chosen","Rejected":"rejected"})

In [9]:
# Convert the DataFrame to a Dataset
dataset1 = Dataset.from_pandas(df)

# To create a DatasetDict, you can simply create a dictionary with your datasets
df_dict = {'my_dataset': dataset1}

In [10]:
df_dict = df_dict['my_dataset'].train_test_split(test_size=0.1)

# Now, dataset_dict is a DatasetDict with 'train' and 'test' datasets
print(df_dict)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 46
    })
    test: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 6
    })
})


In [11]:
orpo_args = ORPOConfig(
    learning_rate=8e-6,
    lr_scheduler_type="linear",
    max_length=1024,
    max_prompt_length=512,
    beta=0.1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    num_train_epochs=1,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    output_dir="./results/",
)

trainer = ORPOTrainer(
    model=model,
    args=orpo_args,
    train_dataset=df_dict["train"],
    eval_dataset=df_dict["test"],
    peft_config=peft_config,
    tokenizer=tokenizer,
)
trainer.train()
trainer.save_model(new_model)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/orpo_trainer.py:247: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/46 [00:00<?, ? examples/s]

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Runtime,Samples Per Second,Steps Per Second,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen,Nll Loss,Log Odds Ratio,Log Odds Chosen
1,2.475000,2.818835,29.116300,0.206000,0.069000,-0.250898,-0.124417,0.125000,-0.126481,-1.244166,-2.508979,-2.994689,-6.097871,2.774413,-2.009529,-1.715463
2,2.918800,2.816706,29.298900,0.205000,0.068000,-0.250615,-0.124285,0.125000,-0.126330,-1.242854,-2.506151,-2.994679,-6.096658,2.772161,-2.008293,-1.714113
3,2.237900,2.812457,29.319100,0.205000,0.068000,-0.250061,-0.124026,0.125000,-0.126034,-1.240261,-2.500606,-2.994618,-6.094223,2.767695,-2.005916,-1.711488


/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:209: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:209: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


In [13]:
# Flush memory
##gc.collect()
#gc.collect()
#torch.cuda.empty_cache()

# Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model)
fp16_model = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
)
fp16_model, tokenizer = setup_chat_format(fp16_model, tokenizer)

new_model="/kaggle/working/OrpoLlama-2-7b"

# Merge adapter with base model
model = PeftModel.from_pretrained(fp16_model, new_model)
model = model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
model.push_to_hub("orpo-base-sql")
#tokenizer.push_to_hub(new_model, use_temp_dir=False)

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Nishika26/orpo-base-sql/commit/d4088a75777b91d4167fe3aabb2f0ff9bd7e73dc', commit_message='Upload LlamaForCausalLM', commit_description='', oid='d4088a75777b91d4167fe3aabb2f0ff9bd7e73dc', pr_url=None, pr_revision=None, pr_num=None)

In [16]:
tokenizer.push_to_hub("orpo-base-sql")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Nishika26/orpo-base-sql/commit/56cb4e5e9b6233dea1a7155bd22935c5d05621c5', commit_message='Upload tokenizer', commit_description='', oid='56cb4e5e9b6233dea1a7155bd22935c5d05621c5', pr_url=None, pr_revision=None, pr_num=None)